# This covers all 3 sections of the assignment.
## 1. Transform the data in the table on the Wikipedia page into the above pandas dataframe.
### Find the required libraries

In [ ]:
!pip install beautifulsoup4
!pip install lxml
import requests
import pandas as pd 
import numpy as np 
import random

from geopy.geocoders import Nominatim

from IPython.display import Image 
from IPython.core.display import HTML 

from IPython.display import display_html
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium 
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

print('All done')

Solving environment: \ 

### Post Codes

In [ ]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(source,'lxml')
print(soup.title)
from IPython.display import display_html
tab = str(soup.table)
display_html(tab,raw=True)

### Make it a Panda

In [ ]:
dfs = pd.read_html(tab)
df=dfs[0]
df.head()

## Cleaning Up

In [ ]:
df1 = df[df.Borough != 'Not assigned']

df2 = df1.groupby(['Postcode','Borough'], sort=False).agg(', '.join)
df2.reset_index(inplace=True)

df2['Neighbourhood'] = np.where(df2['Neighbourhood'] == 'Not assigned',df2['Borough'], df2['Neighbourhood'])

## 2. Create geocoder dataframe

### Get CSV file

In [ ]:
lat_lon = pd.read_csv('https://cocl.us/Geospatial_data')
lat_lon.head()

In [ ]:
lat_lon.rename(columns={'Postal Code':'Postcode'},inplace=True)
df3 = pd.merge(df2,lat_lon,on='Postcode')
df3.head()


## 3. Cluster Toronto Neighbourhoods.

In [ ]:
df4 = df3[df3['Borough'].str.contains('Toronto',regex=False)]
df4

### Cluster 

In [ ]:
k=5
toronto_clustering = df4.drop(['Postcode','Borough','Neighbourhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_clustering)
kmeans.labels_
df4.insert(0, 'Cluster Labels', kmeans.labels_)
df4

### Map

In [ ]:
map_toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

for lat,lng,borough,neighbourhood in zip(df4['Latitude'],df4['Longitude'],df4['Borough'],df4['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto